In [2]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd

In [3]:
from utils import process_from_census_data, TRS

### Import getting model

In [4]:
model = keras.models.load_model("cnn_20.keras")

## Combine to get total EV for each SA1 in the final

### Process data

In [14]:
file_loc = "output/EV_pred"
geo_lev = "POA"

In [11]:
poa_data, poa_tot = process_from_census_data(geo_lev="POA", normalise=False, boxcox=False, keep_same=True, return_tot=True)
electric_vals = poa_data["Fuel type__Electric"]

In [12]:
electric_vals

3000    170.0
3002     32.0
3003     19.0
3004     82.0
3006     56.0
        ...  
3990      0.0
3991      0.0
3992      0.0
3995     14.0
3996     10.0
Name: Fuel type__Electric, Length: 691, dtype: float64

In [15]:
df_all = pd.read_csv(f"{file_loc}/{geo_lev}_EV_pred_all.csv")
df_all[geo_lev] = df_all[geo_lev].astype(str)

In [32]:
dict_num_ev =  {f"POA{poa}": int(val) for poa, val in zip(electric_vals.index, electric_vals)}

ls_zone_not_exist =  []
for z in df_all[geo_lev].unique():
    check = z[3:]
    if check not in electric_vals.index:
        print(check)
        dict_num_ev[f"POA{check}"] = 0 #special case

3644
3691
3707


In [17]:
ls_cols_ev_pred = [x for x in df_all.columns if "EV_pred" in x]

In [18]:
combine_fin_df = df_all[ls_cols_ev_pred+[geo_lev]]

## Assigning EV

In [19]:
combine_fin_df

,EV_pred_forest,EV_pred_baye,EV_pred_lr,EV_pred_GraBoost,EV_pred_Pearsons,POA
0,0.006153,0.011787,-0.016936,0.002052,0.528474,POA3000
1,0.003944,0.010410,0.009832,0.001441,0.339234,POA3000
2,0.003435,0.009044,0.026785,0.001441,0.152534,POA3000
3,0.003180,0.007614,0.012467,0.001441,0.151239,POA3000
4,0.002398,0.009283,0.007622,0.001441,0.062332,POA3000
...,...,...,...,...,...,...
2390809,0.002443,0.001969,0.033027,0.001441,0.035259,POA3996
2390810,0.002146,-0.002589,-0.001190,0.001441,-0.113637,POA3996
2390811,0.005030,0.008055,0.007859,0.001441,0.305439,POA3996
2390812,0.004118,0.000006,-0.023657,0.001441,0.089287,POA3996


In [33]:
# Make sense to have higher, some sa1 does not have households
print(len(dict_num_ev), len(df_all[geo_lev].unique()))

694 692


In [34]:
dict_thres = {}
ls_df = [] # Note these one will have diff index matching with original to combine later

for zone in df_all[geo_lev].unique():
    print(f"DOING {zone}")
    dict_thres[zone] = {}
    sub_df = combine_fin_df[combine_fin_df[geo_lev]==zone]
    num_ev = dict_num_ev[zone]
    ls_series = []
    for ev_pred in ls_cols_ev_pred:
        sort_df = sub_df.sort_values(ev_pred, ascending=False)
        n = len(sort_df)
        arr_val = ["NO" for _ in range(n)]
        arr_val[:num_ev] = ["YES" for _ in range(num_ev)]
        seri_ev_count = pd.Series(arr_val, index=sort_df.index, name=f"check_{ev_pred}")
        ls_series.append(seri_ev_count)

        # Find thres
        thres = sort_df[ev_pred].iat[num_ev-1]
        dict_thres[zone][ev_pred] = thres
    df_zone = pd.concat(ls_series, axis=1) # note the index
    ls_df.append(df_zone)

DOING POA3000
DOING POA3002
DOING POA3003
DOING POA3004
DOING POA3006
DOING POA3008
DOING POA3011
DOING POA3012
DOING POA3013
DOING POA3015
DOING POA3016
DOING POA3018
DOING POA3019
DOING POA3020
DOING POA3021
DOING POA3022
DOING POA3023
DOING POA3024
DOING POA3025
DOING POA3026
DOING POA3027
DOING POA3028
DOING POA3029
DOING POA3030
DOING POA3031
DOING POA3032
DOING POA3033
DOING POA3034
DOING POA3036
DOING POA3037
DOING POA3038
DOING POA3039
DOING POA3040
DOING POA3041
DOING POA3042
DOING POA3043
DOING POA3044
DOING POA3046
DOING POA3047
DOING POA3048
DOING POA3049
DOING POA3051
DOING POA3052
DOING POA3053
DOING POA3054
DOING POA3055
DOING POA3056
DOING POA3057
DOING POA3058
DOING POA3059
DOING POA3060
DOING POA3061
DOING POA3063
DOING POA3064
DOING POA3065
DOING POA3066
DOING POA3067
DOING POA3068
DOING POA3070
DOING POA3071
DOING POA3072
DOING POA3073
DOING POA3074
DOING POA3075
DOING POA3076
DOING POA3078
DOING POA3079
DOING POA3081
DOING POA3082
DOING POA3083
DOING POA3084
DOING 

In [35]:
ev_as_df = pd.concat(ls_df)
ev_as_df = ev_as_df.sort_index()
ev_as_df

,check_EV_pred_forest,check_EV_pred_baye,check_EV_pred_lr,check_EV_pred_GraBoost,check_EV_pred_Pearsons
0,NO,NO,NO,NO,YES
1,NO,NO,NO,NO,NO
2,NO,NO,NO,NO,NO
3,NO,NO,NO,NO,NO
4,NO,NO,NO,NO,NO
...,...,...,...,...,...
2390809,NO,NO,NO,NO,NO
2390810,NO,NO,NO,NO,NO
2390811,NO,NO,NO,NO,NO
2390812,NO,NO,NO,NO,NO


In [36]:
final_df_all = pd.concat([combine_fin_df, ev_as_df], axis=1)
final_df_all

,EV_pred_forest,EV_pred_baye,EV_pred_lr,EV_pred_GraBoost,EV_pred_Pearsons,POA,check_EV_pred_forest,check_EV_pred_baye,check_EV_pred_lr,check_EV_pred_GraBoost,check_EV_pred_Pearsons
0,0.006153,0.011787,-0.016936,0.002052,0.528474,POA3000,NO,NO,NO,NO,YES
1,0.003944,0.010410,0.009832,0.001441,0.339234,POA3000,NO,NO,NO,NO,NO
2,0.003435,0.009044,0.026785,0.001441,0.152534,POA3000,NO,NO,NO,NO,NO
3,0.003180,0.007614,0.012467,0.001441,0.151239,POA3000,NO,NO,NO,NO,NO
4,0.002398,0.009283,0.007622,0.001441,0.062332,POA3000,NO,NO,NO,NO,NO
...,...,...,...,...,...,...,...,...,...,...,...
2390809,0.002443,0.001969,0.033027,0.001441,0.035259,POA3996,NO,NO,NO,NO,NO
2390810,0.002146,-0.002589,-0.001190,0.001441,-0.113637,POA3996,NO,NO,NO,NO,NO
2390811,0.005030,0.008055,0.007859,0.001441,0.305439,POA3996,NO,NO,NO,NO,NO
2390812,0.004118,0.000006,-0.023657,0.001441,0.089287,POA3996,NO,NO,NO,NO,NO


In [37]:
check_cols = [x for x in final_df_all.columns if "check_" in x]

for c in check_cols:
    print(final_df_all[c].value_counts())

check_EV_pred_forest
NO     2382995
YES       7819
Name: count, dtype: int64
check_EV_pred_baye
NO     2382995
YES       7819
Name: count, dtype: int64
check_EV_pred_lr
NO     2382995
YES       7819
Name: count, dtype: int64
check_EV_pred_GraBoost
NO     2382995
YES       7819
Name: count, dtype: int64
check_EV_pred_Pearsons
NO     2382995
YES       7819
Name: count, dtype: int64


In [38]:
final_df_all.to_csv("POA_FIN_EV_assignment.csv", index=False)

# Mapping to get the final plot

In [ ]:
import matplotlib.pyplot as plt